In [1]:
import cv2
import torch
import torchvision
from tqdm import tqdm
import numpy as np
from monai.transforms import (
    Compose, LoadImage, EnsureChannelFirst, ScaleIntensity, HistogramNormalize,
    CropForeground, Zoom, Resize, DivisiblePad, ToTensor, SaveImage
)

In [2]:
%%bash
mkdir -p demo/xr/

In [3]:
import os
import glob
from tqdm.auto import tqdm

In [4]:
def glob_files(folders: str = None, extension: str = "*.nii.gz"):
    assert folders is not None
    paths = [
        glob.glob(os.path.join(folder, extension), recursive=True)
        for folder in folders
    ]
    files = sorted([item for sublist in paths for item in sublist])
    print(len(files))
    print(files[:1])
    return files

In [5]:
xr_folders = [
    "data/ChestXRLungSegmentation/VinDr/v1/processed/test/images/"
]
xr_images = glob_files(
    xr_folders, 
    extension="*.png"
)

3000
['data/ChestXRLungSegmentation/VinDr/v1/processed/test/images/002a34c58c5b758217ed1f584ccbcfe9.png']


In [6]:
# Define the transformation pipeline for "image2d"
val_transforms = Compose(
    [
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        ScaleIntensity(minv=0.0, maxv=255.0),
        HistogramNormalize(min=0.0, max=255.0),
        CropForeground(select_fn=lambda x: x > 0, margin=0),
        Zoom(zoom=0.95, padding_mode="constant", mode="area"),
        Resize(spatial_size=256, size_mode="longest", mode="area"),
        DivisiblePad(k=(256, 256), mode="constant", constant_values=0),
        ToTensor(), 
        SaveImage(
            output_dir="demo/xr/", 
            output_postfix='', 
            output_ext='.png',
            separate_folder=False,
        )
    ]
)

/home/quantm/anaconda3/envs/py310/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.array CropForeground.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)


In [7]:
for image in tqdm(xr_images):
    sample = val_transforms(image)

  0%|          | 0/3000 [00:00<?, ?it/s]

2024-09-01 16:47:35,861 INFO image_writer.py:197 - writing: demo/xr/002a34c58c5b758217ed1f584ccbcfe9.png
2024-09-01 16:47:36,546 INFO image_writer.py:197 - writing: demo/xr/004f33259ee4aef671c2b95d54e4be68.png
2024-09-01 16:47:37,095 INFO image_writer.py:197 - writing: demo/xr/008bdde2af2462e86fd373a445d0f4cd.png
2024-09-01 16:47:37,484 INFO image_writer.py:197 - writing: demo/xr/009bc039326338823ca3aa84381f17f1.png
2024-09-01 16:47:37,966 INFO image_writer.py:197 - writing: demo/xr/00a2145de1886cb9eb88869c85d74080.png
2024-09-01 16:47:38,787 INFO image_writer.py:197 - writing: demo/xr/00b7e6bfa4dc1fe9ddd0ce74743e38c2.png
2024-09-01 16:47:39,199 INFO image_writer.py:197 - writing: demo/xr/011295e0bcdc7636569ab73bfdcc4450.png
2024-09-01 16:47:39,765 INFO image_writer.py:197 - writing: demo/xr/013c169f9dad6f1f6485da961b9f7bf2.png
2024-09-01 16:47:40,324 INFO image_writer.py:197 - writing: demo/xr/01431a2618c0ace741e4e270a37e20b9.png
2024-09-01 16:47:40,915 INFO image_writer.py:197 - writ